### Probability Map  Generation
This file provides direct access to model outputs for downstream analysis.

In [14]:
from os.path import join
from gelgenie.segmentation.evaluation.core_functions import segment_and_analyze, model_predict_and_process
from gelgenie.segmentation.helper_functions.general_functions import create_dir_if_empty
import toml
import torch
from gelgenie.segmentation.networks import model_configure
from gelgenie.segmentation.helper_functions.stat_functions import load_statistics
from torch.utils.data import DataLoader
from gelgenie.segmentation.data_handling.dataloaders import ImageDataset, ImageMaskDataset
from tqdm.notebook import tqdm
import numpy as np

def model_eval_load(exp_folder, eval_epoch):

    model_config = toml.load(join(exp_folder, 'config.toml'))['model']
    model, _, _ = model_configure(**model_config)
    if eval_epoch == 'best':
        stats = load_statistics(join(exp_folder, 'training_logs'), 'training_stats.csv', config='pd')
        sel_epoch = stats['Epoch'][stats['Dice Score'].idxmax()]
    else:
        sel_epoch = eval_epoch

    checkpoint = torch.load(f=join(exp_folder, 'checkpoints', 'checkpoint_epoch_%s.pth' % sel_epoch),
                            map_location=torch.device("cpu"))
    model.load_state_dict(checkpoint['network'])
    model.eval()

    return model



In [2]:
experiment_names = ['unet_global_padding_nov_4']
eval_epochs = [198]
model_folder = '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/segmentation_models/November 2023'
output_folder = '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/probability_map_samples'
input_data = '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/probability_map_samples/true_data'
images_folder = join(input_data, 'images')
masks_folder = join(input_data, 'masks')
create_dir_if_empty(output_folder)

models = []
for experiment, eval_epoch in zip(experiment_names, eval_epochs):
    exp_folder = join(model_folder, experiment)
    model = model_eval_load(exp_folder, eval_epoch)
    models.append(model)

In [3]:
# testing direct model outputs
segment_and_analyze(models, experiment_names, images_folder, output_folder)

Created dataset with 6 images.

100%|██████████| 6/6 [00:33<00:00,  5.66s/it]


In [15]:
dataset = ImageMaskDataset(images_folder, masks_folder, 1, padding=False, individual_padding=True)
dataloader = DataLoader(dataset, shuffle=False, batch_size=1, num_workers=0, pin_memory=True)
model_outputs = join(output_folder, 'direct_model_outputs')
create_dir_if_empty(model_outputs)

for im_index, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    np_mask = batch['mask'].detach().squeeze().cpu().numpy()
    name = batch['image_name'][0]
    all_model_outputs = []
    sel_image_folder = join(model_outputs, name)
    create_dir_if_empty(sel_image_folder)
    
    seg_mask, seg_ordered_mask = model_predict_and_process(models[0], batch['image'])
    np.save(join(sel_image_folder, 'seg_mask.npy'), seg_mask.detach().squeeze().cpu().numpy())
    np.save(join(sel_image_folder, 'seg_mask_one_hot.npy'), seg_ordered_mask)
    np.save(join(sel_image_folder, 'true_mask.npy'), np_mask)

    

Class weighting is [0.51268754] for background, [20.20436883] for bands

Created dataset with 6 images.

/var/folders/4x/_gjrrkj918v37xmw7d8n_p2h0000gn/T/ipykernel_3993/1272120084.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for im_index, batch in tqdm(enumerate(dataloader), total=len(dataloader)):


  0%|          | 0/6 [00:00<?, ?it/s]